In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 102kB 18.2MB/s ta 0:00:01
Libraries imported.


# 2. Scraping Data

In [3]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
neighborhoodList = []

In [6]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [8]:
kl_df.shape

(40, 1)

# 3.Get the Geographical Coordinates

In [11]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [12]:
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

coords

[[19.11848309908247, 72.84177419095158],
 [19.042830000000038, 72.92734000000007],
 [19.06293000000005, 72.92666000000008],
 [19.054220000000043, 72.84019000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.062200000000075, 72.90242000000006],
 [19.250030000000038, 72.85908000000006],
 [19.224720000000048, 72.86606000000006],
 [19.212870000000066, 73.08324000000005],
 [19.00538889189226, 72.85576887678867],
 [19.086476606699875, 72.9089562772808],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.13790000000006, 72.84941000000003],
 [19.01493000000005, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21195211212422, 72.83754191243007],
 [19.131400000000042, 72.93565000000007],
 [19.127560000000074, 72.82540000000006],
 [19.064940000000036, 72.88073000000003],
 [19.21195211212422, 72.83754191243007],
 [19.048530000000028, 72.93220000000008],


In [13]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [15]:
print(kl_df.shape)
kl_df.head()

(40, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118483,72.841774
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062930,72.926660
3,Bandra,19.054220,72.840190
4,Bhandup,19.145560,72.948560


In [16]:
kl_df.to_csv("kl_df.csv", index=False)

# 4. Create a map of Mumbai with neighbourhoods superimposed on top

In [17]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [18]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl


In [20]:
map_kl.save('map_kl.jpeg')

# 5. Use the Foursquare API to explore the neighborhoods

In [21]:
CLIENT_ID = '22F3WDAIGKTOSTREQM0JDUUXQFOY0IQOPNZGN1ADG2QSMUJ1' # your Foursquare ID
CLIENT_SECRET = 'SUYTQ22WV155ZY5IJPD2Q4VZAKJC0RAICY2N225RGN4I3XUC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 22F3WDAIGKTOSTREQM0JDUUXQFOY0IQOPNZGN1ADG2QSMUJ1
CLIENT_SECRET:SUYTQ22WV155ZY5IJPD2Q4VZAKJC0RAICY2N225RGN4I3XUC


In [22]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2635, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118483,72.841774,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118483,72.841774,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118483,72.841774,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118483,72.841774,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop
4,Andheri,19.118483,72.841774,Shawarma Factory,19.124591,72.840398,Falafel Restaurant


In [24]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,19,19,19,19,19,19
Baiganwadi,9,9,9,9,9,9
Bandra,100,100,100,100,100,100
Bhandup,25,25,25,25,25,25
Borivali,97,97,97,97,97,97
Charkop,53,53,53,53,53,53
Chembur,99,99,99,99,99,99
Dahisar,57,57,57,57,57,57


In [25]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 203 uniques categories.


In [26]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop', 'Coffee Shop',
       'Falafel Restaurant', 'Fast Food Restaurant', 'Pub', 'Café',
       'Pizza Place', 'Juice Bar', 'Multiplex', 'Snack Place',
       'Breakfast Spot', 'Sandwich Place', 'Cocktail Bar', 'Bar',
       'American Restaurant', 'Chinese Restaurant', 'Seafood Restaurant',
       'Electronics Store', 'Diner', 'Department Store',
       'Asian Restaurant', 'Vegetarian / Vegan Restaurant', 'Spa',
       'BBQ Joint', 'Residential Building (Apartment / Condo)', 'Lounge',
       'Gym / Fitness Center', "Women's Store", 'Liquor Store',
       'Smoke Shop', 'Food Truck', 'Athletics & Sports', 'Fish Market',
       'Hotel', 'Martial Arts Dojo', 'Tea Room', 'Clothing Store', 'Food',
       'Plaza', 'Sports Bar', 'Park', 'Platform', 'Food & Drink Shop',
       'Music Venue', 'Hot Dog Joint', 'Tunnel', 'Fried Chicken Joint',
       'Dessert Shop'], dtype=object)

In [27]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

# 6.Analyze each neighbourhood

In [28]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2635, 204)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,High School,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [29]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(40, 204)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,High School,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Outdoors & Recreation,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0.0,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.020000,0.010000,0.010000,0.020000,0.000000,0.040000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.050000,0.010000,0.01,0.020000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.000000,0.020000,0.000000,0.020000,0.000000,0.0,0.000000,0.010000,0.00,0.000000,0.010000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.040000,0.170000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.010000,0.020000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0

In [30]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

16

new data frame for shopping mall only

In [31]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [32]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.00
1,Anushakti Nagar,0.00
2,Baiganwadi,0.00
3,Bandra,0.00
4,Bhandup,0.04


# 7. Cluster Neighbourhoods

In [33]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 1, 0, 0, 0, 1], dtype=int32)

In [34]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [35]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.00,0
1,Anushakti Nagar,0.00,0
2,Baiganwadi,0.00,0
3,Bandra,0.00,0
4,Bhandup,0.04,2


In [36]:
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(40, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.00,0,19.118483,72.841774
1,Anushakti Nagar,0.00,0,19.042830,72.927340
2,Baiganwadi,0.00,0,19.062930,72.926660
3,Bandra,0.00,0,19.054220,72.840190
4,Bhandup,0.04,2,19.145560,72.948560


In [37]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(40, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118483,72.841774
26,Mogra Village,0.000000,0,26.135300,73.054840
25,Mira Road,0.000000,0,19.074161,72.861670
24,"Matharpacady, Mumbai",0.000000,0,19.044920,72.867205
23,Mankhurd,0.000000,0,19.048530,72.932200
22,Mahavir Nagar (Kandivali),0.000000,0,19.211952,72.837542
21,Kurla,0.000000,0,19.064940,72.880730
20,Kausa,0.000000,0,19.127560,72.825400
30,Seven Bungalows,0.010000,0,19.128560,72.820850
18,Kandivali,0.000000,0,19.211952,72.837542


## Visualising resultant clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
map_clusters.save('map_clusters.html')


# 8. Examine Clusters

## Cluster 0

In [40]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118483,72.841774
26,Mogra Village,0.000000,0,26.135300,73.054840
25,Mira Road,0.000000,0,19.074161,72.861670
24,"Matharpacady, Mumbai",0.000000,0,19.044920,72.867205
23,Mankhurd,0.000000,0,19.048530,72.932200
22,Mahavir Nagar (Kandivali),0.000000,0,19.211952,72.837542
21,Kurla,0.000000,0,19.064940,72.880730
20,Kausa,0.000000,0,19.127560,72.825400
30,Seven Bungalows,0.010000,0,19.128560,72.820850
18,Kandivali,0.000000,0,19.211952,72.837542


## Cluster 1

In [41]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
29,Pestom sagar,0.023810,1,19.070630,72.902200
36,Vikhroli,0.030000,1,19.111090,72.927810
37,Wadala,0.021277,1,19.017160,72.858130
39,Worli,0.020000,1,19.007450,72.816910
38,Western Suburbs (Mumbai),0.011905,1,19.197000,72.827630
13,Goregaon,0.020000,1,19.164550,72.849460
12,Ghatkopar,0.025316,1,19.086477,72.908956
9,Devipada,0.022472,1,19.224720,72.866060
5,Borivali,0.020619,1,19.229360,72.857510


## Cluster 2

In [42]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
33,Thakur village,0.048387,2,19.21020,72.87541
35,Vashi,0.040000,2,19.08470,72.90484
4,Bhandup,0.040000,2,19.14556,72.94856
27,Mulund,0.040541,2,19.17185,72.95564
19,Kanjurmarg,0.038462,2,19.13140,72.93565


# Observations

From the above result, it is obvious that most of the shooping malls are present in cluster 2 neighbourhoods. The malls here face extreme competition. But since there are no to very few malls in cluster 0 and cluster 1 neighbourhoods,the malls built there would face no or very less competition. So, the ideal places to build new malls are neighbourhoods of cluter 0 and cluster 1. 